# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [12]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [13]:

city_weather_data = pd.read_csv("../output_data/city_weather_data.csv")
city_weather_data


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,itoman,26.12,127.67,22.00,64,40,5.14,JP,1615692746
1,boa vista,2.82,-60.67,25.00,83,0,1.03,BR,1615692679
2,hermanus,-34.42,19.23,17.22,95,99,2.32,ZA,1615692747
3,albany,42.60,-73.97,0.00,47,1,2.06,US,1615692560
4,busselton,-33.65,115.33,21.11,51,0,1.34,AU,1615692582
...,...,...,...,...,...,...,...,...,...
543,livingston,55.90,-3.52,4.00,81,3,5.66,GB,1615692854
544,xichang,27.90,102.26,21.49,37,0,2.14,CN,1615692854
545,saint-raymond,45.47,-73.61,-2.78,80,100,1.54,CA,1615692855
546,sinnamary,5.38,-52.95,24.99,89,100,5.59,GF,1615692855


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [14]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [23]:
locations = city_weather_data[['Lat', 'Lng']]
humidity= city_weather_data['Humidity']

# Create a Humidity Heatmap layer
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100,point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [16]:
#A max temperature lower than 80 degrees but higher than 70.Wind speed less than 10 mph. Zero cloudiness.

city_weather_data['Max Temp'] = [(C * 9/5) + 32 for C in  city_weather_data['Max Temp']]

narrow_data_df = city_weather_data.loc[(city_weather_data["Wind Speed"] <= 10) & (city_weather_data["Cloudiness"] == 0) & \
                                   (city_weather_data['Max Temp'] >=  70) & (city_weather_data['Max Temp'] <= 80)].dropna()

narrow_data_df


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,boa vista,2.82,-60.67,77.000,83,0,1.03,BR,1615692679
61,richards bay,-28.78,32.04,71.312,86,0,4.71,ZA,1615692620
85,ngunguru,-35.62,174.50,75.992,59,0,1.79,NZ,1615692761
104,kharan,28.58,65.42,70.448,29,0,3.17,PK,1615692677
114,saint-philippe,-21.36,55.77,75.992,73,0,3.09,RE,1615692589
119,tomatlan,19.93,-105.25,70.124,51,0,1.23,MX,1615692722
139,east london,-33.02,27.91,71.600,83,0,3.09,ZA,1615692593
164,saint-pierre,-21.34,55.48,75.992,73,0,3.09,RE,1615692660
182,cockburn town,21.46,-71.14,75.506,75,0,9.53,TC,1615692650
252,saint-joseph,-21.37,55.62,75.992,73,0,3.09,RE,1615692669


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [17]:
hotel_df =narrow_data_df.loc[:,['City','Country','Lat','Lng']]
hotel_df["Hotel Name"] = " "
print(hotel_df)

               City Country    Lat     Lng Hotel Name
1         boa vista      BR   2.82  -60.67           
61     richards bay      ZA -28.78   32.04           
85         ngunguru      NZ -35.62  174.50           
104          kharan      PK  28.58   65.42           
114  saint-philippe      RE -21.36   55.77           
119        tomatlan      MX  19.93 -105.25           
139     east london      ZA -33.02   27.91           
164    saint-pierre      RE -21.34   55.48           
182   cockburn town      TC  21.46  -71.14           
252    saint-joseph      RE -21.37   55.62           
308        beroroha      MG -21.67   45.17           
309           mongo      TD  12.18   18.69           
355      le vauclin      MQ  14.55  -60.84           
381           pisco      PE -13.70  -76.22           
397    buenos aires      AR -34.61  -58.38           
414        la plata      AR -34.92  -57.95           
441      rock sound      BS  24.90  -76.20           
468          maceio      BR 

In [18]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}


# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "HOtels" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
        
    
    

Missing field/result... skipping.


In [19]:
print(hotel_df)

               City Country    Lat     Lng  \
1         boa vista      BR   2.82  -60.67   
61     richards bay      ZA -28.78   32.04   
85         ngunguru      NZ -35.62  174.50   
104          kharan      PK  28.58   65.42   
114  saint-philippe      RE -21.36   55.77   
119        tomatlan      MX  19.93 -105.25   
139     east london      ZA -33.02   27.91   
164    saint-pierre      RE -21.34   55.48   
182   cockburn town      TC  21.46  -71.14   
252    saint-joseph      RE -21.37   55.62   
308        beroroha      MG -21.67   45.17   
309           mongo      TD  12.18   18.69   
355      le vauclin      MQ  14.55  -60.84   
381           pisco      PE -13.70  -76.22   
397    buenos aires      AR -34.61  -58.38   
414        la plata      AR -34.92  -57.95   
441      rock sound      BS  24.90  -76.20   
468          maceio      BR  -9.67  -35.74   
482      san isidro      PE -12.12  -77.05   
544         xichang      CN  27.90  102.26   

                                 

In [20]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]
print(locations)

       Lat     Lng
1     2.82  -60.67
61  -28.78   32.04
85  -35.62  174.50
104  28.58   65.42
114 -21.36   55.77
119  19.93 -105.25
139 -33.02   27.91
164 -21.34   55.48
182  21.46  -71.14
252 -21.37   55.62
308 -21.67   45.17
309  12.18   18.69
355  14.55  -60.84
381 -13.70  -76.22
397 -34.61  -58.38
414 -34.92  -57.95
441  24.90  -76.20
468  -9.67  -35.74
482 -12.12  -77.05
544  27.90  102.26


In [21]:
# Add marker layer and info box content ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))